# Intro

{Fill in with information about this notebook}

# Set Up notebook 

In [1]:
#Import modules
import pandas as pd
import csv
import numpy as np

#The rest of this is just for fun/quick dataviz at the end to map and ensure seems about right
import cartopy
import matplotlib.pyplot as plt
import rasterio
import geopandas

from lib import readData
from lib import mapping
from lib import cleanData
from lib import classify
from lib import exportData

todayDate, dateSuffix = readData.getCurrentDate()

2023-01-09


datetime.date(2023, 1, 9)

# Read in data

- Set up filepaths
- Read in data from:
    - downholeData table (from database)
    - headerData table (from database)
    - xyzData file (from previously carried out work) (will eventually make this updateable)

Read in data

In [4]:
downholeDataPATH, headerDataPATH, xyzInPATH  = readData.filesSetup()

#Functions to read data into dataframes. Also excludes extraneous columns
downholeDataIN, headerDataIN = readData.readRawTxtData(downholefile=downholeDataPATH, headerfile=headerDataPATH)
xyzDataIN = readData.readXYZData(xyzfile=xyzInPATH)

UFuncTypeError: ufunc 'subtract' cannot use operands with types dtype('<M8[D]') and dtype('float64')

Define datatypes (doing this during the read in process has presented issues)

In [ ]:
#Define datatypes, to read into defineDataTypes() function
downholeDataDTYPES = {'ID':np.uint32, "API_NUMBER":np.uint64,"TABLE_NAME":str,"WHO":str,"INTERPRET_DATE":str,"FORMATION":str,"THICKNESS":np.float64,"TOP":np.float64,"BOTTOM":np.float64}
headerDataDTYPES = {'ID':np.uint32,'API_NUMBER':np.uint64,"TDFORMATION":str,"PRODFORM":str,"TOTAL_DEPTH":np.float64,"SECTION":np.float64,"TWP":np.float64,"TDIR":str,"RNG":np.float64,"RDIR":str,"MERIDIAN":np.float64,"FARM_NAME":str,"NSFOOT":np.float64,"NSDIR":str,"EWFOOT":np.float64,"EWDIR":str,"QUARTERS":str,"ELEVATION":np.float64,"ELEVREF":str,"COMP_DATE":str,"STATUS":str,"FARM_NUM":str,"COUNTY_CODE":np.float64,"PERMIT_NUMBER":str,"COMPANY_NAME":str,"COMPANY_CODE":str,"PERMIT_DATE":str,"CORNER":str,"LATITUDE":np.float64,"LONGITUDE":np.float64,"ENTERED_BY":str,"UPDDATE":str,"ELEVSOURCE":str, "ELEV_FT":np.float64}
xyzDataDTYPES = {'ID':np.uint64, 'API_NUMBER':np.uint64, "LATITUDE":np.float64, "LONGITUDE":np.float64, "ELEV_FT":np.float64}

#Define datatypes of each column of the new dataframes
downholeDataIN = readData.defineDataTypes(downholeDataIN, downholeDataDTYPES)
headerDataIN = readData.defineDataTypes(headerDataIN, headerDataDTYPES)
xyzDataIN = readData.defineDataTypes(xyzDataIN, xyzDataDTYPES)

#Make a copy of the data so raw data is preserved while we work with the rest of the data
downholeData = downholeDataIN.copy()
headerData = headerDataIN.copy()
xyzData = xyzDataIN.copy()

# Extract Elevation Data

In [ ]:
#statewideLidar =  ow
#mapping.rastertoPoints_extract()

## Extract elevation data from consistent elevation dataset for entire dataset

In [71]:
uniqueWells = headerData['API_NUMBER'].unique()
xyzData['UniqueWells'] = uniqueWells

headerDataXYZ = mapping.addElevtoHeader(xyzData, headerData)

In [59]:
headerOutFile = 'headerData__essCols_lidarElev.csv'

encodeType='latin-1'

processDirectory = '\\\\isgs-sinkhole\\geophysics\\Balikian\\ISWS_HydroGeo\\WellDataAutoClassification\\ProcessedData\\'
headerXYZData.to_csv(processDirectory+headerOutFile,index_label='ID')


# Data Cleaning

## First, let's clean up records in the data without the necessary information

First, the headerfileData had elevations from the 10m statewide DEM added to get a consistent surface elevation data source (I would like to do this with lidar data, but that is much more involved)

Start by reading the files back in again

In [60]:
headerInFile = headerOutFile #'headerData__essCols_lidarElev.csv'
downholeInFile = downholeOutFile #'downholeData_essCols.csv'

encodeType='latin-1'

processDirectory = '\\\\isgs-sinkhole\\geophysics\\Balikian\\ISWS_HydroGeo\\WellDataAutoClassification\\ProcessedData\\'

headers_useCols = ['API_NUMBER',"LATITUDE","LONGITUDE","ELEV_FT","TOTAL_DEPTH","SECTION","TWP","TDIR","RNG","RDIR","MERIDIAN","QUARTERS","ELEVATION","ELEVREF","COUNTY_CODE"]
downhole_useCols = ["API_NUMBER","TABLE_NAME","FORMATION","THICKNESS","TOP","BOTTOM"]

headerDataIN = pd.read_csv(processDirectory+headerInFile, sep=',', header='infer', encoding=encodeType, dtype = headerDataDTYPES, usecols=headers_useCols)
downholeDataIN = pd.read_csv(processDirectory+downholeInFile, sep=',', header='infer', encoding=encodeType, usecols=downhole_useCols)

In [61]:
headerData = headerDataIN.copy()
downholeData = downholeDataIN.copy()
headerData

,API_NUMBER,TOTAL_DEPTH,SECTION,TWP,TDIR,RNG,RDIR,MERIDIAN,QUARTERS,ELEVATION,ELEVREF,COUNTY_CODE,LATITUDE,LONGITUDE,ELEV_FT
0,120352191101,1400.0,15.0,10.0,N,10.0,E,3.0,SW SW,611.0,GL,35.0,NaN,NaN,NaN
1,120352194900,4183.0,27.0,9.0,N,9.0,E,3.0,SE SW,584.0,KB,35.0,NaN,NaN,NaN
2,120352204200,2140.0,18.0,10.0,N,11.0,E,3.0,NE NE,652.0,KB,35.0,NaN,NaN,NaN
3,120352204300,4175.0,34.0,10.0,N,9.0,E,3.0,NE NE,592.0,KB,35.0,NaN,NaN,NaN
4,120352207500,2461.0,34.0,9.0,N,10.0,E,3.0,NW SW,582.0,GL,35.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
834523,120810182400,24.0,3.0,4.0,S,4.0,E,3.0,SESE,455.0,NaN,81.0,NaN,NaN,NaN
834524,120810182500,59.0,25.0,3.0,S,3.0,E,3.0,SENE,528.0,NaN,81.0,NaN,NaN,NaN
834525,120810244500,711.0,9.0,4.0,S,2.0,E,3.0,NENW,400.0,NaN,81.0,NaN,NaN,NaN
834526,120810245100,518.0,20.0,3.0,S,1.0,E,3.0,NW,512.0,NaN,81.0,NaN,NaN,NaN


Define datatypes for all columns

In [ ]:
downholeDataDTYPES = {"ID":np.uint32, "API_NUMBER":np.uint64,"TABLE_NAME":str,"WHO":str,"INTERPRET_DATE":str,"FORMATION":str,"THICKNESS":np.float64,"TOP":np.float64,"BOTTOM":np.float64}
headerDataDTYPES = {"ID":np.uint32, 'API_NUMBER':np.uint64,"TDFORMATION":str,"PRODFORM":str,"TOTAL_DEPTH":np.float64,"SECTION":np.float64,"TWP":np.float64,"TDIR":str,"RNG":np.float64,"RDIR":str,"MERIDIAN":np.float64,"FARM_NAME":str,"NSFOOT":np.float64,"NSDIR":str,"EWFOOT":np.float64,"EWDIR":str,"QUARTERS":str,"ELEVATION":np.float64,"ELEVREF":str,"COMP_DATE":str,"STATUS":str,"FARM_NUM":str,"COUNTY_CODE":np.float64,"PERMIT_NUMBER":str,"COMPANY_NAME":str,"COMPANY_CODE":str,"PERMIT_DATE":str,"CORNER":str,"LATITUDE":np.float64,"LONGITUDE":np.float64,"ENTERED_BY":str,"UPDDATE":str,"ELEVSOURCE":str, "10mDTM_ft":np.float64}

#Recast all columns to datatypes of downholeData to previously defined types
for i in range(0, np.shape(downholeData)[1]):
    downholeData.iloc[:,i] = downholeDataIN.iloc[:,i].astype(downholeDataDTYPES[downholeDataIN.iloc[:,i].name])
    
#Recast all columns to datatypes of headerData to previously defined types
for i in range(0, np.shape(headerData)[1]):
    headerData.iloc[:,i] = headerDataIN.iloc[:,i].astype(headerDataDTYPES[headerDataIN.iloc[:,i].name])

Remove headerData rows without Lat/lon information

In [ ]:
#Read in headerData file
before = headerDataIN.shape[0]
headerDataIN = headerDataIN.dropna(subset=['LATITUDE'])
headerDataIN = headerDataIN.dropna(subset=['LONGITUDE'])
after = headerDataIN.shape[0]
headerData = headerDataIN.copy()
headerDataIN
print(str(before-after)+' records removed without Lat/Lon')

0 records removed without Lat/Lon


Remove downholeData rows without associated Lat/Lon

In [ ]:
before = downholeData.shape[0]
df = pd.merge(downholeData, headerData.set_index('API_NUMBER'), on='API_NUMBER', how='left', indicator='Exist')
df['Existbool'] = np.where(df['Exist'] == 'both', True, False)
df = df[df['Existbool']==True].drop(['Exist','Existbool'], axis=1)
downholeData = df[['API_NUMBER','TABLE_NAME','FORMATION','THICKNESS','TOP','BOTTOM']].copy()
after = downholeData.shape[0]
print(str(before-after)+' records removed without Lat/Lon')

11440 records removed without Lat/Lon


Remove headerData rows without surface elevation information (this also clips data from outside Illinois)

In [ ]:
before = headerData.shape[0]
print("Number of rows before dropping those without surface elevation information: "+str(headerData.shape[0]))
headerData['10mDTM_ft'].replace('',np.nan,inplace=True)
headerData.dropna(subset=['10mDTM_ft'],inplace=True)
print("Number of rows after dropping those without surface elevation information: "+str(headerData.shape[0]))
print('Well records deleted: '+str(before-headerData.shape[0]))


Number of rows before dropping those without surface elevation information: 634755
Number of rows after dropping those without surface elevation information: 634533
Well records deleted: 222


Remove rows from downholeData with no depth information and where depth information is obviously bad (i.e., top depth > bottom depth)

In [ ]:
#Replace empty cells in top/bottom columns with nans
downholeData['TOP'] = downholeData['TOP'].replace('', np.nan)
downholeData['BOTTOM'] = downholeData['BOTTOM'].replace('', np.nan)
before = downholeData.shape[0]
print("Number of rows before dropping those without record depth information: "+str(downholeDataIN.shape[0]))

#Drop records without depth information
downholeData = downholeData.dropna(subset=['TOP'])
downholeData = downholeData.dropna(subset=['BOTTOM'])
print("Number of rows after dropping those without record depth information: "+str(downholeData.shape[0]))
print('Well records deleted: '+str(before-downholeData.shape[0]))
before = downholeData.shape[0]

#Replace empty cells in formation column with nans
downholeData['FORMATION'] = downholeData['FORMATION'].replace('', np.nan) 
before = downholeData.shape[0]
print("Number of rows before dropping those without FORMATION information: "+str(downholeData.shape[0]))

#Drop records without FORMATION information
downholeData = downholeData.dropna(subset=['FORMATION'])
print("Number of rows after dropping those without FORMATION information: "+str(downholeData.shape[0]))
print('Well records deleted: '+str(before-downholeData.shape[0]))

#Calculate thickness (replaces existing column) and drop records with obviously bad depth information
downholeData['THICKNESS'] = downholeData['BOTTOM'] - downholeData['TOP']
before = downholeData.shape[0]
downholeData = downholeData[(downholeData['THICKNESS'] >= 0)]
print("Number of rows before dropping those with obviously bad depth information: "+str(before))
print("Number of rows after dropping those with obviously bad depth information: "+str(downholeData.shape[0]))
print('Well records deleted: '+str(before-downholeData.shape[0]))
downholeData

Number of rows before dropping those without record depth information: 3017085
Number of rows after dropping those without record depth information: 2590844
Well records deleted: 414801
Number of rows before dropping those without FORMATION information: 2590844
Number of rows after dropping those without FORMATION information: 2590844
Well records deleted: 0
Number of rows before dropping those with obviously bad depth information: 2590844
Number of rows after dropping those with obviously bad depth information: 2588277
Well records deleted: 2567


,API_NUMBER,TABLE_NAME,FORMATION,THICKNESS,TOP,BOTTOM
6,120010000300,WFORMATIONS,"Colchester No. 2 coal,Penn. carb",2.0,18.0,20.0
7,120010000300,WFORMATIONS,fire clay at,0.0,20.0,20.0
8,120010000300,WFORMATIONS,interval drift?,18.0,0.0,18.0
14,120010000800,WFORMATIONS,coal No. 2 possibly at,0.0,175.0,175.0
15,120010000800,WFORMATIONS,interval,175.0,0.0,175.0
...,...,...,...,...,...,...
3016997,480590043000,WFORMATIONS,"red clay, firm",120.0,45.0,165.0
3016998,480590043000,WFORMATIONS,sand silt & gravel,165.0,6.0,171.0
3016999,480590043000,WFORMATIONS,"sandy blue clay, soft",85.0,35.0,120.0
3017000,480590043000,WFORMATIONS,stoney blue clay,18.0,67.0,85.0


Now we are going to export this data, again to save memory and space and so we do not lose data

In [ ]:
headerOutFile = 'headerData__essCols_elev10mClip.csv'
downholeOutFile = 'downholeData_essCols_depthNA.csv'

encodeType='latin-1'

processDirectory = '\\\\isgs-sinkhole\\geophysics\\Balikian\\ISWS_HydroGeo\\WellDataAutoClassification\\ProcessedData\\'

In [ ]:
headerData.reset_index(inplace=True,drop=True)
downholeData.reset_index(inplace=True,drop=True)

headerData.to_csv(processDirectory+headerOutFile,index_label='ID')
downholeData.to_csv(processDirectory+downholeOutFile,index_label='ID')

PermissionError: [Errno 13] Permission denied: '\\\\isgs-sinkhole\\geophysics\\Balikian\\ISWS_HydroGeo\\WellDataAutoClassification\\ProcessedData\\headerData__essCols_elev10mClip.csv'

# Data Merging

First, read data back in again

In [ ]:
#File names
headerInFile = 'headerData__essCols_elev10mClip.csv'
downholeInFile = 'downholeData_essCols_depthNA.csv'

encodeType='latin-1'

#Directory path
processDirectory = '\\\\isgs-sinkhole\\geophysics\\Balikian\\ISWS_HydroGeo\\WellDataAutoClassification\\ProcessedData\\'

#Columns to use
headers_useCols = ['ID','API_NUMBER',"TOTAL_DEPTH","SECTION","TWP","TDIR","RNG","RDIR","MERIDIAN","QUARTERS","ELEVATION","ELEVREF","COUNTY_CODE","LATITUDE","LONGITUDE","ELEVSOURCE"]
downhole_useCols = ["ID","API_NUMBER","TABLE_NAME","FORMATION","THICKNESS","TOP","BOTTOM"]

#Read it in
headerDataIN = pd.read_csv(processDirectory+headerInFile, sep=',', header='infer', encoding=encodeType, index_col='ID',usecols=headers_useCols)
downholeDataIN = pd.read_csv(processDirectory+downholeInFile, sep=',', header='infer', encoding=encodeType, index_col='ID',usecols=downhole_useCols)

#Make a copy, in case we need to refer back to original read-in data
headerData = headerDataIN.copy()
downholeData = downholeDataIN.copy()

In [ ]:
downholeDataDTYPES = {"ID":np.uint32, "API_NUMBER":np.uint64,"TABLE_NAME":str,"WHO":str,"INTERPRET_DATE":str,"FORMATION":str,"THICKNESS":np.float64,"TOP":np.float64,"BOTTOM":np.float64}
headerDataDTYPES = {"ID":np.uint32, 'API_NUMBER':np.uint64,"TDFORMATION":str,"PRODFORM":str,"TOTAL_DEPTH":np.float64,"SECTION":np.float64,"TWP":np.float64,"TDIR":str,"RNG":np.float64,"RDIR":str,"MERIDIAN":np.float64,"FARM_NAME":str,"NSFOOT":np.float64,"NSDIR":str,"EWFOOT":np.float64,"EWDIR":str,"QUARTERS":str,"ELEVATION":np.float64,"ELEVREF":str,"COMP_DATE":str,"STATUS":str,"FARM_NUM":str,"COUNTY_CODE":np.float64,"PERMIT_NUMBER":str,"COMPANY_NAME":str,"COMPANY_CODE":str,"PERMIT_DATE":str,"CORNER":str,"LATITUDE":np.float64,"LONGITUDE":np.float64,"ENTERED_BY":str,"UPDDATE":str,"ELEVSOURCE":str}

#Recast all columns to datatypes of downholeData to previously defined types
for i in range(0, np.shape(downholeData)[1]):
    downholeData.iloc[:,i] = downholeDataIN.iloc[:,i].astype(downholeDataDTYPES[downholeDataIN.iloc[:,i].name])
    
#Recast all columns to datatypes of headerData to previously defined types
for i in range(0, np.shape(headerData)[1]):
    headerData.iloc[:,i] = headerDataIN.iloc[:,i].astype(headerDataDTYPES[headerDataIN.iloc[:,i].name])

# Processing

The following flags are used to mark the classification method:
- 0: Not classified
- 1: Specific Search Term Match
- 2: Wildcard match (startTerm) - no context
- 3: Bedrock classification for obvious bedrock
- 4: Wildcard match (startTerm) - with context

In [ ]:
#Read in dictionary files for downhole data
dictDir = "\\\\isgs-sinkhole\\geophysics\\Balikian\\ISWS_HydroGeo\\WellDataAutoClassification\\SupportingDocs\\"

specTermsFile = "SearchTerms-Specific.csv" #Specific matches
startTermsFile = "SearchTerms-Start.csv" #Wildcard matches for the start of the description

In [ ]:
specTerms = pd.read_csv(dictDir+specTermsFile, index_col='ID')
startTerms = pd.read_csv(dictDir+startTermsFile, index_col='ID')
specTerms.rename(columns={'SearchTerm':'FORMATION', 'InterpUpdate':'INTERPRETATION'}, inplace=True)
specTerms['CLASS_FLAG'] = 1
startTerms.rename(columns={'StartTerm':'FORMATION', 'InterpUpdate':'INTERPRETATION'}, inplace=True)
startTerms['CLASS_FLAG'] = 2
specTerms

,FORMATION,INTERPRETATION,CLASS_FLAG
ID,,,
0.0,"sand, fine gr",SAND,1
1.0,"sand, fine-large gr",SAND,1
2.0,"sand, some large gr",SAND,1
3.0,toop soil,SOIL,1
4.0,grav in muck,GRAVEL,1
...,...,...,...
2013.0,yellow sandstone at,BEDROCK,1
2014.0,yellow sandstone/clay,BEDROCK AND OTHER,1
2015.0,yellow St. Peter,BEDROCK,1


In [ ]:
specTermsDtypes = {'FORMATION':str,'INTERPRETATION':str, 'CLASS_FLAG':np.uint8}
startTermsDtypes = {'FORMATION':str,'INTERPRETATION':str, 'CLASS_FLAG':np.uint8}

#Recast all columns to datatypes of headerData to previously defined types
for i in range(0, np.shape(specTerms)[1]):
    specTerms.iloc[:,i] = specTerms.iloc[:,i].astype(specTermsDtypes[specTerms.iloc[:,i].name])
    startTerms.iloc[:,i] = startTerms.iloc[:,i].astype(startTermsDtypes[startTerms.iloc[:,i].name])
specTerms.drop_duplicates(subset='FORMATION',inplace=True) #Apparently, there are some duplicate definitions, which need to be deleted first
startTerms.drop_duplicates(subset='FORMATION',inplace=True)
specTerms.reset_index(inplace=True, drop=True)
startTerms.reset_index(inplace=True, drop=True)


Join the dataframes--for the specific search terms, this is the same as classifying them

In [ ]:
downholeData_Interps = pd.merge(downholeData, specTerms.set_index('FORMATION'), on='FORMATION',how='left')
print("Records Classified with full search term: "+str(int(downholeData_Interps['CLASS_FLAG'].sum())))
print("Records Classified with full search term: "+str(round((downholeData_Interps['CLASS_FLAG'].sum()/downholeData_Interps.shape[0])*100,2))+"% of data")
downholeData_Interps

Records Classified with full search term: 1161561
Records Classified with full search term: 44.88% of data


,API_NUMBER,TABLE_NAME,FORMATION,THICKNESS,TOP,BOTTOM,INTERPRETATION,CLASS_FLAG
0,120010000300,WFORMATIONS,"Colchester No. 2 coal,Penn. carb",2.0,18.0,20.0,NaN,NaN
1,120010000300,WFORMATIONS,fire clay at,0.0,20.0,20.0,NaN,NaN
2,120010000300,WFORMATIONS,interval drift?,18.0,0.0,18.0,NaN,NaN
3,120010000800,WFORMATIONS,coal No. 2 possibly at,0.0,175.0,175.0,NaN,NaN
4,120010000800,WFORMATIONS,interval,175.0,0.0,175.0,NaN,NaN
...,...,...,...,...,...,...,...,...
2588272,480590043000,WFORMATIONS,"red clay, firm",120.0,45.0,165.0,NaN,NaN
2588273,480590043000,WFORMATIONS,sand silt & gravel,165.0,6.0,171.0,NaN,NaN
2588274,480590043000,WFORMATIONS,"sandy blue clay, soft",85.0,35.0,120.0,NaN,NaN
2588275,480590043000,WFORMATIONS,stoney blue clay,18.0,67.0,85.0,NaN,NaN


Create a dataframe with only the records classified using the specific search terms

In [ ]:
specDF = downholeData_Interps[downholeData_Interps['CLASS_FLAG'].notna()]
specDF

,API_NUMBER,TABLE_NAME,FORMATION,THICKNESS,TOP,BOTTOM,INTERPRETATION,CLASS_FLAG
5,120010000900,WFORMATIONS,coal,2.0,133.0,135.0,BEDROCK,1.0
6,120010000900,WFORMATIONS,fire clay,1.0,135.0,136.0,BEDROCK,1.0
9,120010001000,WFORMATIONS,fire clay,2.0,137.0,139.0,BEDROCK,1.0
16,120010001200,WFORMATIONS,black slate,1.0,35.0,36.0,BEDROCK,1.0
17,120010001200,WFORMATIONS,coal,1.0,36.0,37.0,BEDROCK,1.0
...,...,...,...,...,...,...,...,...
2588263,480590027200,WFORMATIONS,yellow clay,1.0,25.0,26.0,CLAY,1.0
2588268,480590028100,WFORMATIONS,sandy clay,127.0,39.0,166.0,SAND AND CLAY MIX,1.0
2588269,480590028100,WFORMATIONS,topsoil,0.0,1.0,1.0,SOIL,1.0
2588270,480590028100,WFORMATIONS,yellow clay,1.0,21.0,22.0,CLAY,1.0


Create a dataframe that should be mutually exclusive from the specDF that we will use to search within for the 'wildcard' terms

We create a separate dataframe so we do not overwrite our previous (better) specific term classification

In [ ]:
searchDF = downholeData_Interps[downholeData_Interps['CLASS_FLAG'].isna()]
searchDF

,API_NUMBER,TABLE_NAME,FORMATION,THICKNESS,TOP,BOTTOM,INTERPRETATION,CLASS_FLAG
0,120010000300,WFORMATIONS,"Colchester No. 2 coal,Penn. carb",2.0,18.0,20.0,NaN,NaN
1,120010000300,WFORMATIONS,fire clay at,0.0,20.0,20.0,NaN,NaN
2,120010000300,WFORMATIONS,interval drift?,18.0,0.0,18.0,NaN,NaN
3,120010000800,WFORMATIONS,coal No. 2 possibly at,0.0,175.0,175.0,NaN,NaN
4,120010000800,WFORMATIONS,interval,175.0,0.0,175.0,NaN,NaN
...,...,...,...,...,...,...,...,...
2588271,480590043000,WFORMATIONS,clean sand & fine gravel,171.0,5.0,176.0,NaN,NaN
2588272,480590043000,WFORMATIONS,"red clay, firm",120.0,45.0,165.0,NaN,NaN
2588273,480590043000,WFORMATIONS,sand silt & gravel,165.0,6.0,171.0,NaN,NaN
2588274,480590043000,WFORMATIONS,"sandy blue clay, soft",85.0,35.0,120.0,NaN,NaN


Now, do the classification routine on the searchDF database

In [ ]:
for i,s in enumerate(startTerms['FORMATION']):
    searchDF['CLASS_FLAG'].where(~searchDF['FORMATION'].str.startswith(s,na=False),4,inplace=True)
    searchDF['INTERPRETATION'].where(~searchDF['FORMATION'].str.startswith(s,na=False),startTerms.loc[i,'INTERPRETATION'],inplace=True)

C:\Users\balikian\AppData\Local\Temp\ipykernel_28376\1840067340.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  searchDF['CLASS_FLAG'].where(~searchDF['FORMATION'].str.startswith(s,na=False),4,inplace=True)
C:\Users\balikian\AppData\Local\Temp\ipykernel_28376\1840067340.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  searchDF['INTERPRETATION'].where(~searchDF['FORMATION'].str.startswith(s,na=False),startTerms.loc[i,'INTERPRETATION'],inplace=True)


In [ ]:
print("Records classified with start search term: "+str(int(searchDF['CLASS_FLAG'].count())))
print("Records classified with start search term: "+str(round((searchDF['CLASS_FLAG'].count()/searchDF.shape[0])*100,2))+"% of remaining data")
print("Records classified with both search terms: "+str(round(((searchDF['CLASS_FLAG'].count()+specDF['CLASS_FLAG'].count())/downholeData_Interps.shape[0])*100,2))+"% of all data")

Records classified with start search term: 167146
Records classified with start search term: 11.72% of remaining data
Records classified with both search terms: 51.34% of all data


In [ ]:
#Just checking some data out of interest
searchDF.INTERPRETATION.value_counts()

BEDROCK                126816
CLAY                    20678
SAND AND GRAVEL          7795
SAND                     4861
SOIL                     1630
SILT                     1284
FILL                     1117
CLAY AND SAND MIX         732
BOULDERY MATERIAL         711
ORGANIC MATERIAL          398
CLAY AND GRAVEL           274
GENERIC                   235
SAND AND CLAY MIX         160
SILT AND SAND             146
GRAVEL                     93
SILT AND SAND MIX          87
CLAY WITH SAND             80
BEDROCK AND OTHER          22
TOPSOIL                    12
MUD AND GRAVEL             11
MUD AND SAND                2
GRAVEL WITH SAND            1
SILT AND GRAVEL MIX         1
Name: INTERPRETATION, dtype: int64

In [ ]:
#Just checking random interval to make sure it all looks ok
searchDF.iloc[100:120,:]

,API_NUMBER,TABLE_NAME,FORMATION,THICKNESS,TOP,BOTTOM,INTERPRETATION,CLASS_FLAG
128,120010002400,WFORMATIONS,ironstone,1.0,115.0,116.0,NaN,NaN
129,120010002400,WFORMATIONS,"sandstone,soft",50.0,65.0,115.0,BEDROCK,4.0
130,120010002500,WFORMATIONS,blue clay,60.0,60.0,120.0,NaN,NaN
136,120010002700,WFORMATIONS,coal no.2,3.0,55.0,58.0,NaN,NaN
138,120010002700,WFORMATIONS,interval,37.0,0.0,37.0,NaN,NaN
141,120010002900,WFORMATIONS,"c w/ucl,sft,s,lgt gry",3.0,35.0,38.0,NaN,NaN
142,120010002900,WFORMATIONS,"s,most f,slgt sty,noncalc,yl-brn",2.0,13.0,15.0,NaN,NaN
143,120010002900,WFORMATIONS,"s,most f,sty,noncalc,yl-brn",3.0,10.0,13.0,NaN,NaN
144,120010002900,WFORMATIONS,"s,most f,vy ltl st,noncalc,ylsh",2.0,18.0,20.0,NaN,NaN
145,120010002900,WFORMATIONS,"s,most f,vy ltl st,noncalc,ylsh",3.0,15.0,18.0,NaN,NaN


Merge specDF and searchDF back together all back in single dataframe

In [ ]:
downholeData_Cat = pd.concat([specDF,searchDF], join='inner').sort_index()
downholeData_Cat

,API_NUMBER,TABLE_NAME,FORMATION,THICKNESS,TOP,BOTTOM,INTERPRETATION,CLASS_FLAG
0,120010000300,WFORMATIONS,"Colchester No. 2 coal,Penn. carb",2.0,18.0,20.0,NaN,NaN
1,120010000300,WFORMATIONS,fire clay at,0.0,20.0,20.0,NaN,NaN
2,120010000300,WFORMATIONS,interval drift?,18.0,0.0,18.0,NaN,NaN
3,120010000800,WFORMATIONS,coal No. 2 possibly at,0.0,175.0,175.0,NaN,NaN
4,120010000800,WFORMATIONS,interval,175.0,0.0,175.0,NaN,NaN
...,...,...,...,...,...,...,...,...
2588272,480590043000,WFORMATIONS,"red clay, firm",120.0,45.0,165.0,NaN,NaN
2588273,480590043000,WFORMATIONS,sand silt & gravel,165.0,6.0,171.0,NaN,NaN
2588274,480590043000,WFORMATIONS,"sandy blue clay, soft",85.0,35.0,120.0,NaN,NaN
2588275,480590043000,WFORMATIONS,stoney blue clay,18.0,67.0,85.0,NaN,NaN


In [ ]:
downholeData = downholeData_Cat.copy()

## Classify all data deeper than 550' as bedrock

First, separate into data that still needs to be classified and data that already has been classified

In [ ]:
#same as above, but now it is foundDF instead of specDF since we have multiple searches
searchDF = downholeData[downholeData['CLASS_FLAG'].isna()] #Data that has not yet been classified
foundDF = downholeData[downholeData['CLASS_FLAG'].notna()] #Data that has already been classified
searchDF

,API_NUMBER,TABLE_NAME,FORMATION,THICKNESS,TOP,BOTTOM,INTERPRETATION,CLASS_FLAG
0,120010000300,WFORMATIONS,"Colchester No. 2 coal,Penn. carb",2.0,18.0,20.0,NaN,NaN
1,120010000300,WFORMATIONS,fire clay at,0.0,20.0,20.0,NaN,NaN
2,120010000300,WFORMATIONS,interval drift?,18.0,0.0,18.0,NaN,NaN
3,120010000800,WFORMATIONS,coal No. 2 possibly at,0.0,175.0,175.0,NaN,NaN
4,120010000800,WFORMATIONS,interval,175.0,0.0,175.0,NaN,NaN
...,...,...,...,...,...,...,...,...
2588271,480590043000,WFORMATIONS,clean sand & fine gravel,171.0,5.0,176.0,NaN,NaN
2588272,480590043000,WFORMATIONS,"red clay, firm",120.0,45.0,165.0,NaN,NaN
2588273,480590043000,WFORMATIONS,sand silt & gravel,165.0,6.0,171.0,NaN,NaN
2588274,480590043000,WFORMATIONS,"sandy blue clay, soft",85.0,35.0,120.0,NaN,NaN


In [ ]:
searchDF['CLASS_FLAG'].mask(searchDF['TOP']>550,3,inplace=True) #Add a Classification Flag of 3 (bedrock b/c it's deepter than 550') to all records where the top of the interval is >550'
#searchDF['BEDROCK_FLAG'].mask(searchDF['TOP']>550,True,inplace=True) #Not using this 

prevClass = downholeData['CLASS_FLAG'].value_counts().sum()
brDepthClass = searchDF['CLASS_FLAG'].value_counts()[3.0]
total = downholeData.shape[0]

newPercClass = round((brDepthClass/(total-prevClass))*100, 2)

print("Records classified deeper than 550': " + str(brDepthClass))
print("Records classified deeper than 550', as a percentage of remaining unclassified records: " + str(newPercClass)+"%")
print("Total bedrock records classified with search terms and depth: " + str(prevClass+brDepthClass))
print("This is now a total of "+str(round((prevClass+brDepthClass)*100/downholeData.shape[0],2))+"% of the data classified.")



Records classified deeper than 550': 273243
Records classified deeper than 550', as a percentage of remaining unclassified records: 21.69%
Total bedrock records classified with search terms and depth: 1601950
This is now a total of 61.89% of the data classified.


C:\Users\balikian\AppData\Local\Temp\ipykernel_28376\3883727400.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  searchDF['CLASS_FLAG'].mask(searchDF['TOP']>550,3,inplace=True) #Add a Classification Flag of 3 (bedrock b/c it's deepter than 550') to all records where the top of the interval is >550'


Put the classified and unclassified data back together again

In [ ]:
downholeData_Cat = pd.concat([foundDF,searchDF], join='inner').sort_index()

In [ ]:
downholeData = downholeData_Cat.copy() #make a copy of the data to continue further work on

Add '0' flag for data still not classified

In [ ]:
downholeData['CLASS_FLAG'].fillna(0, inplace=True)

In [ ]:
downholeData['CLASS_FLAG'].value_counts()

1.0    1161561
0.0     986327
3.0     273243
4.0     167146
Name: CLASS_FLAG, dtype: int64

## Add "Flag" for target interpratations

In [ ]:
#dictDir = "\\\\isgs-sinkhole\\geophysics\\Balikian\\ISWS_HydroGeo\\WellDataAutoClassification\\SupportingDocs\\"
InterpFile = 'Lithology_Interp_FineCoarse.csv'
targetInterpDF = pd.read_csv(dictDir+InterpFile)
targetInterpDF.rename(columns={'LITHOLOGY':'INTERPRETATION', 'CODE':'TARGET'}, inplace=True)
targetInterpDF

,INTERPRETATION,TARGET
0,BEDROCK,DoNotUse
1,FIRECLAY,DoNotUse
2,VOID,DoNotUse
3,CLAY AND STONE,0
4,DIRT AND BEDROCK,DoNotUse
5,DIRT AND STONE,DoNotUse
6,GRAVEL AND STONE,1
7,BOULDER,1
8,BOULDERS AND CLAY,0
9,GRAVEL WITH BOULDERS,1


In [ ]:
downholeData_targ = pd.merge(downholeData, targetInterpDF.set_index('INTERPRETATION'), right_on='INTERPRETATION',left_on='INTERPRETATION', how='left')
downholeData = downholeData_targ.copy()
downholeData['TARGET'].replace('DoNotUse', value=-1, inplace=True)
downholeData['TARGET'].fillna(value=-2, inplace=True)
downholeData['TARGET'].astype(np.int8)
downholeData

,API_NUMBER,TABLE_NAME,FORMATION,THICKNESS,TOP,BOTTOM,INTERPRETATION,CLASS_FLAG,TARGET
0,120010000300,WFORMATIONS,"Colchester No. 2 coal,Penn. carb",2.0,18.0,20.0,NaN,0.0,-2
1,120010000300,WFORMATIONS,fire clay at,0.0,20.0,20.0,NaN,0.0,-2
2,120010000300,WFORMATIONS,interval drift?,18.0,0.0,18.0,NaN,0.0,-2
3,120010000800,WFORMATIONS,coal No. 2 possibly at,0.0,175.0,175.0,NaN,0.0,-2
4,120010000800,WFORMATIONS,interval,175.0,0.0,175.0,NaN,0.0,-2
...,...,...,...,...,...,...,...,...,...
2588272,480590043000,WFORMATIONS,"red clay, firm",120.0,45.0,165.0,NaN,0.0,-2
2588273,480590043000,WFORMATIONS,sand silt & gravel,165.0,6.0,171.0,NaN,0.0,-2
2588274,480590043000,WFORMATIONS,"sandy blue clay, soft",85.0,35.0,120.0,NaN,0.0,-2
2588275,480590043000,WFORMATIONS,stoney blue clay,18.0,67.0,85.0,NaN,0.0,-2


Flags used for target classification purposes:
- -2: No classification 
- -1: Classified, not used/not definitive
- 0: Classified, not target material
- 1: Classified as target material

In [ ]:
downholeData['TARGET'].value_counts()

-2    1337019
-1     722307
0      336977
1      191974
Name: TARGET, dtype: int64

Find all unique wells in downhole dataset

In [ ]:
#Get Unique well APIs
uniqueWells = downholeData['API_NUMBER'].unique()
wellsDF = pd.DataFrame(uniqueWells)
print('Number of unique wells in downholeData: '+str(wellsDF.shape[0]))
wellsDF.columns = ['UNIQUE_API']
wellsDF

Number of unique wells in downholeData: 413902


,UNIQUE_API
0,120010000300
1,120010000800
2,120010000900
3,120010001000
4,120010001100
...,...
413897,480590000900
413898,480590001200
413899,480590027200
413900,480590028100


Sort dataset by API Number and Depth of top of record (will be easier to do data analysis with records in the correct order)

In [ ]:
downholeData_sorted = downholeData.sort_values(['API_NUMBER','TOP'])
downholeData_sorted.reset_index(inplace=True)
downholeData_sorted

,index,API_NUMBER,TABLE_NAME,FORMATION,THICKNESS,TOP,BOTTOM,INTERPRETATION,CLASS_FLAG,TARGET
0,2,120010000300,WFORMATIONS,interval drift?,18.0,0.0,18.0,NaN,0.0,-2
1,0,120010000300,WFORMATIONS,"Colchester No. 2 coal,Penn. carb",2.0,18.0,20.0,NaN,0.0,-2
2,1,120010000300,WFORMATIONS,fire clay at,0.0,20.0,20.0,NaN,0.0,-2
3,4,120010000800,WFORMATIONS,interval,175.0,0.0,175.0,NaN,0.0,-2
4,3,120010000800,WFORMATIONS,coal No. 2 possibly at,0.0,175.0,175.0,NaN,0.0,-2
...,...,...,...,...,...,...,...,...,...,...
2588272,2588273,480590043000,WFORMATIONS,sand silt & gravel,165.0,6.0,171.0,NaN,0.0,-2
2588273,2588276,480590043000,WFORMATIONS,yellow clay,0.0,18.0,18.0,CLAY,1.0,0
2588274,2588274,480590043000,WFORMATIONS,"sandy blue clay, soft",85.0,35.0,120.0,NaN,0.0,-2
2588275,2588272,480590043000,WFORMATIONS,"red clay, firm",120.0,45.0,165.0,NaN,0.0,-2


# Get Bedrock Depth and Layer Thickness

Reproject and align raster grids for surface elevation and bedrock topo (reproject well data too if needed)

In [ ]:
###TO DO 


Use the surface elevation raster and bedrock elevation raster to get depth to bedrock

In [ ]:
###TO DO 

Now, divide by 9 to get thickness of layer at each point in model grid

In [ ]:
##TO DO 

Now, sample each well point (headerData) to get layer thickness, surface elevation, and bedrock 

In [ ]:
#TO DO
#PRevious code that might be useful
#xyzData = pd.read_csv(processDirectory+"headerData__essCols_elev10mDTM.csv",usecols = ['API_NUMBER', 'LONGITUDE','LATITUDE', '10mDTM_ft'])
#downhole_XYZ = pd.merge(downholeData_sorted, xyzData.set_index('API_NUMBER'), on='API_NUMBER',how='left')
#downhole_XYZ['TOP_ELEV_FT'] = downhole_XYZ['10mDTM_ft'] - downhole_XYZ['TOP']
#downhole_XYZ['BOT_ELEV_FT'] = downhole_XYZ['10mDTM_ft'] - downhole_XYZ['BOTTOM']
#downhole_XYZ['TOP_ELEV_M'] = downhole_XYZ['TOP_ELEV_FT'] * 0.3048
#downhole_XYZ['BOT_ELEV_M'] = downhole_XYZ['BOT_ELEV_FT'] * 0.3048
#downhole_XYZ

Calculate  all layer depths/elevations at all wells

In [ ]:
#TO DO

Define function to calculate target thickness in each layer

In [ ]:
##THIS CELL MAY NEED TO BE UPDATED!!!!!


#Define the function to export the result of thickness of target sediments in each layer
def Layers_surfacedown(df, layer = 1):
    
    #Generate Column names based on (looped) integers
    topCol = "ESL_ModelTopoLyrs_"+str(layer)
    if layer != 9: #For all layers except the bottom layer....
        botCol = "ESL_ModelTopoLyrs_"+str(layer+1) #use the layer below it to 
    else: #Otherwise, ...
        botCol = "BedrockCorr" #Use the (corrected) bedrock depth

    #Divide records into 4 separate categories for ease of calculation, to be joined back together later  
        #Category 1: Well interval starts above layer top, ends within model layer
        #Category 2: Well interval is entirely contained withing model layer
        #Category 3: Well interval starts within model layer, continues through bottom of model layer
        #Category 4: well interval begins and ends on either side of model layer (model layer is contained within well layer)

    #records1 = intervals that go through the top of the layer and bottom is within layer
    records1 = df.loc[(df['TOP_ELEV_ft'] > df[topCol]) & (df['BOT_ELEV_ft'] > df[botCol]) & (df['BOT_ELEV_ft'] <= df[topCol]) & (df['BOT_ELEV_ft'] <= df['TOP_ELEV_ft'])].copy()
    records1['TARG_THICK'] = pd.DataFrame(np.round((records1.loc[:,topCol]-records1.loc[: , 'BOT_ELEV_ft']) * records1['Target'],3)).copy()
    
    #records2 = entire interval is within layer
    records2 = df.loc[(df['TOP_ELEV_ft'] <= df[topCol]) & (df['BOT_ELEV_ft'] >= df[botCol]) & (df['BOT_ELEV_ft'] <= df['TOP_ELEV_ft'])].copy()
    records2['TARG_THICK'] = pd.DataFrame(np.round((records2.loc[: , 'TOP_ELEV_ft'] - records2.loc[: , 'BOT_ELEV_ft']) * records2['Target'],3)).copy()

    #records3 = intervals with top within layer and bottom of interval going through bottom of layer
    records3 = df.loc[(df['TOP_ELEV_ft'] > df[botCol]) & (df['BOT_ELEV_ft'] < df[botCol]) & (df['TOP_ELEV_ft'] <= df[topCol]) & (df['BOT_ELEV_ft'] <= df['TOP_ELEV_ft'])].copy()
    records3['TARG_THICK'] = pd.DataFrame(np.round((records3.loc[: , 'TOP_ELEV_ft'] - (records3.loc[:,botCol]))*records3['Target'],3)).copy()

    #records4 = interval goes through entire layer
    records4 = df.loc[(df['TOP_ELEV_ft'] > df[topCol]) & (df['BOT_ELEV_ft'] < df[botCol]) & (df['BOT_ELEV_ft'] <= df['TOP_ELEV_ft'])].copy()
    records4['TARG_THICK'] = pd.DataFrame(np.round((records4.loc[: , topCol]-records4.loc[: , botCol]) * records4['Target'],3)).copy()

    
    #Put the four calculated record categories back together into single dataframe
    res = records1.append(records2).append(records3).append(records4)
    
    res_df = res.groupby(['API_NUMBER','LATITUDE','LONGITUDE'],as_index=False).sum()#calculate thickness for each well interval in the layer indicated (e.g., if there are two well intervals from same well in one model layer)

    res_df['TARG_THICK_PER'] = pd.DataFrame(np.round(res_df['TARG_THICK']/res_df['LyrThick'],3)) #Calculate thickness as percent of total layer thickness
    res_df["LAYER"] = layer #Just to have as part of the output file, include the present layer in the file itself as a separate column
    res_df = res_df[['API_NUMBER', 'LATITUDE', 'LONGITUDE', 'TOP', 'BOTTOM','SURF_ELEV_ft', 'TOP_ELEV_ft', 'BOT_ELEV_ft',topCol,botCol,'LyrThick','TARG_THICK', 'TARG_THICK_PER', 'LAYER']].copy() #Format dataframe for output
    
    return res, res_df

Now run that function over all the layers, looping through each one

In [ ]:
#THIS CELL WILL NEED TO BE UPDATED

outDIR = "\\\\isgs-sinkhole\\geophysics\\Balikian\\ISWS_HydroGeo\\MetroEast_HydroGeo\\CodeOutput\\"+codeTarget+"\\"

for i in np.arange(1,10):
    res, res_df = Layers_surfacedown(df, layer = i)#Run the function defined above for each layer
    outputname = codeTargShort+'Lyr'+str(i)+'.csv' #Create a filename based on the layer and target
    res_df.to_csv(outDIR+outputname)  #Export the file to csv
    #Could also potentially save these to variables for use in following cells

NameError: name 'codeTarget' is not defined

# Interpolate thickness values in each layer

Loop through each layer and interpolate (use same parameters (?))

Ensure rasters align (are co-registered) with grid

# Export

In [ ]:
#Export data 
downhole_bedrockDepth_XYZ.to_csv('\\\\isgs-sinkhole\\geophysics\\Balikian\\BedrockWellData\\Wells\\ProcessedWellData\\Downhole_BedrockPicks.csv',index_label="ID")
wPermits_XYZ.to_csv('\\\\isgs-sinkhole\\geophysics\\Balikian\\BedrockWellData\\Wells\\ProcessedWellData\\wPermits_BedrockPicks.csv',index_label="ID")